# How to fine-tune a pre-trained model for the task of paraphrase identification. (Pytorch version)

All the pre-trained transformer models can be fine-tuned for specific tasks such as (such as text classification, NER, text summarization, etc). 
In this notebook, we will learn how to fine-tune a pre-trained model dor the task of paraphrase identification. In this task, the goal is to detect if a pair of sentences have the similar meaning or not. 

For example, this pair of sentence is a pharafrase:
- Symptoms of influenza include fever and nasal congestion.
- A stuffy nose and elevated temperature are signs you may have the flu.

However, the following pair of sentence is not a paraphrase: 
- The incidence of flu is increasing this year.
- Symptoms of influenza include fever and nasal congestion.


## Loading the dataset
Machine Learning (ML) models usually require a large amount of training data to obtain good results. Fortunately, Hugging Face already provides multiple datasets for many different NLP tasks. This link shows all datasets available in Hugging Face: https://huggingface.co/datasets
HuggingFace provides APIs to easily load these models from the hub. 


In our case, for the task of paraphrase identification, we will use the MRPC (Microsoft Research Paraphrase Corpus) dataset [1]. This corpus contains 5,801 pairs of sentences. Each pair is label with a 1, if they are a paraphrase (both sentences have similar meanings) or with 0, otherwise. 

We’ve selected it for this tutorial because it’s a small dataset, so it’s easy to experiment with training on it.

[1] Dolan, B., & Brockett, C. (2005, January). Automatically constructing a corpus of sentential paraphrases. In Third International Workshop on Paraphrasing (IWP2005).

We must install the library dataset:

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 31.6 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 212 kB 70.2 MB/s 
     |████████████████████████████████| 1.1 MB 32.1 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 140 kB 73.4 MB/s 
     |████████████████████████████████| 596 kB 56.1 MB/s 
     |████████████████████████████████| 127 kB 76.1 MB/s 
     |████████████████████████████████| 271 kB 53.6 MB/s 
     |████████████████████████████████| 144 kB 71.9 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 112 kB 71.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing in

Now, we can load the dataset:

In [2]:
from datasets import load_dataset
# glue is a benchmark to measure the performance of ML models in 10 different text classification tasks.
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

We can see that the dataset includes three different subdatasets: training (with 3,368 pairs of sentences), validation (with 400 pairs of sentences) and test (with 1,725 pair of sentences). 

Moreover, each datasest contains several features (fields):
- sentence1: text of the first sentence
- sentence2: text of the second sentence
- label: 1 if both sentences are a paraphrase (that is, they have similar meanings) or 0 otherwise. 
- idx: is the identifier of the pair.


It is possible to inspect these features to know, for example, the type of each feature or  to know, for example, what the values of the label feature represent: 

In [3]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset.features


{'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None)}

Let's show some examples. First, we show positive examples, that is, some pairs of sentences that are labeled as paraphrase


In [4]:
raw_train_dataset[7]

{'idx': 7,
 'label': 1,
 'sentence1': 'The DVD-CCA then appealed to the state Supreme Court .',
 'sentence2': 'The DVD CCA appealed that decision to the U.S. Supreme Court .'}

In [5]:
raw_train_dataset[0]


{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

Now, we will see some negative examples (pairs of sentences that were annotated as non-paraphrase:

In [6]:
raw_train_dataset[15]


{'idx': 16,
 'label': 0,
 'sentence1': 'Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .',
 'sentence2': 'Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .'}

In [7]:
raw_train_dataset[500]

{'idx': 559,
 'label': 0,
 'sentence1': 'The conservancy helped in the purchase of the land from the Estate of Samuel Mills Damon for $ 22 million .',
 'sentence2': 'The park service purchased the land from the estate of Samuel Mills Damon for $ 22 million .'}

In my opinion, some examples may be ambiguous:

In [8]:
raw_train_dataset[10]


{'idx': 11,
 'label': 0,
 'sentence1': 'Legislation making it harder for consumers to erase their debts in bankruptcy court won overwhelming House approval in March .',
 'sentence2': 'Legislation making it harder for consumers to erase their debts in bankruptcy court won speedy , House approval in March and was endorsed by the White House .'}

**Exercise:** 

- Look at element 5 of the validation set, what is its label?
- Look at element 15 of the test set, what is its label?


In [9]:
raw_val_dataset = raw_datasets["validation"]
raw_val_dataset[5]



{'idx': 50,
 'label': 1,
 'sentence1': 'Wal-Mart said it would check all of its million-plus domestic workers to ensure they were legally employed .',
 'sentence2': 'It has also said it would review all of its domestic employees more than 1 million to ensure they have legal status .'}

In [10]:
raw_test_dataset = raw_datasets["test"]
raw_test_dataset[15]


{'idx': 15,
 'label': 0,
 'sentence1': 'Ballmer has been vocal in the past warning that Linux is a threat to Microsoft .',
 'sentence2': 'In the memo , Ballmer reiterated the open-source threat to Microsoft .'}

## Preprocessing

To train our model, first, we need to convert text to numbers. 
To do this, we will use a tokenizer provided by the BERT base model. 

First, we need to install the library transformers. This library provides us the AutoTokenizer class to load the pre-trained tokenizer from any pre-trained model. 
In our example, we will load the tokenizer of the bert-base-uncased model. 

- BERT was pretrained on BookCorpus (11,038 unpublished books and English Wikipedia). 
- uncased means that all texts were lowercased. 
- base: 12-layer, 768-hidden, 12-heads, 110M parameters. 

The maximum input length is 512 tokens. 

Note about uncased versus cased:: 
- In most NLP applications, BERT uncased is the most appropiate option when the case information is not relevant. However, BERT cased is better for NER or PoS taggging, where case information is important.



In [11]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 31.5 MB/s 
     |████████████████████████████████| 6.6 MB 51.9 MB/s 


In [12]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased" # weights of the model BERT base
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Let's to apply this tokenizer on a simple sentence:

In [13]:
inputs = tokenizer("Today, we are learning how to fine-tuning a pre-trained model for paraphrase identification.")
inputs


{'input_ids': [101, 2651, 1010, 2057, 2024, 4083, 2129, 2000, 2986, 1011, 17372, 1037, 3653, 1011, 4738, 2944, 2005, 11498, 8458, 23797, 8720, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

We can see that the tokenizer returns a dictionary. Its first key is "input_ids" that contain a list of the identifiers (integers) of each word in the sentence. For example:
- 101 refers to the special token [CLS], which indicates the beggining of a sentence.
- 102 refers to the special token [SEP], which indicates the end of a sentence.
- 2651 refers to the token 'today'. That is, the identifier of "today" in the vocabulary of the bert-base-uncased model is 2651. 
- 1010 refers to the token ','. 
- 2056 is the identifier of the token 'we'. 


The function **convert_ids_to_tokens** gets a sequence of ids, and returns the tokens associated to these ids.

In [14]:
print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))


['[CLS]', 'today', ',', 'we', 'are', 'learning', 'how', 'to', 'fine', '-', 'tuning', 'a', 'pre', '-', 'trained', 'model', 'for', 'para', '##ph', '##rase', 'identification', '.', '[SEP]']


**Exercise:** What is the identifier of 'Technology'?
What is the identifier of 'technology'?

Hint: the tokenizer can be applied on a sequence of a single token

Solution: 2974

In [15]:
inputs = tokenizer("Technology")
print(inputs)
inputs = tokenizer("technology")
print(inputs)


{'input_ids': [101, 2974, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}
{'input_ids': [101, 2974, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}


**Exercise:** The tokenizer assigns several identifiers to the word 'paraphrase':  11498, 8458, 23797. Why?


In [16]:
inputs = tokenizer("paraphrase")
print(inputs)
print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))


{'input_ids': [101, 11498, 8458, 23797, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}
['[CLS]', 'para', '##ph', '##rase', '[SEP]']


Now, let's apply the tokenizer to the entire dataset at once.
Remember that we have two texts in each instance.
So, we can apply tokenization as follows:

In [17]:
# tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_1 = tokenizer(raw_train_dataset["sentence1"])
tokenized_sentences_2 = tokenizer(raw_train_dataset["sentence2"])


However, this would not be correct. We need to handle the two sequences as a pair, and apply the appropriate preprocessing. 

Fortunately, the tokenizer can also take a pair of sequences and prepare it the way our BERT model expects:



In [18]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs


{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))


['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']


If we align the tokens with the token_type_ids, we can see that the tokens of the first sentence are labeled as 0, while the tokens of the second sentences are labeled with 1.

Note: Some models like DistilBERT model does not have token_type_ids (because they were not trained for the task next sentence prediction).


In [20]:
print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))
print(inputs["token_type_ids"])


['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


As we know that the tokenizer can handle pair of sentences, we can apply it on our whole training dataset. The parameteres padding and trunction helps us for all sequences have the same length


In [21]:
tokenized_train_dataset = tokenizer(
    #raw_datasets["train"]["sentence1"],
    raw_train_dataset["sentence1"],
    #raw_datasets["train"]["sentence2"],
    raw_train_dataset["sentence2"],
    padding=True,
    truncation=True,
)

The tokenizer, in this case, returns a dictionary with three keys: input_ids, token_type_ids, and attention_mask (as before),  but now, the values of these keys are list of lists. 


In [22]:
tokenized_train_dataset

Output hidden; open in https://colab.research.google.com to view.


It makes difficult to handle the instances.  Moreover, ** It will also only work if you have enough RAM to store your whole dataset during the tokenization**

In [23]:
print(raw_train_dataset["sentence1"][0])
print(raw_train_dataset["sentence2"][0])

Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .


In [24]:
print(tokenized_train_dataset['input_ids'][0])
print(tokenizer.convert_ids_to_tokens(tokenized_train_dataset['input_ids'][0]))
print(tokenized_train_dataset['token_type_ids'][0])
print(tokenized_train_dataset['attention_mask'][0])

[101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['[CLS]', 'am', '##ro', '##zi', 'accused', 'his', 'brother', ',', 'whom', 'he', 'called', '"', 'the', 'witness', '"', ',', 'of', 'deliberately', 'di', '##stor', '##ting', 'his', 'evidence', '.', '[SEP]', 'referring', 'to', 'him', 'as', 'only', '"', 'the', 'witness', '"', ',', 'am', '##ro', '##zi', 'accused', 'his', 'brother', 'of', 'deliberately', 'di', '##stor', '##ting', 'his', 'evidence', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [25]:
print(raw_train_dataset["sentence1"][1])
print(tokenized_train_dataset['input_ids'][1])
print(tokenizer.convert_ids_to_tokens(tokenized_train_dataset['input_ids'][1]))

Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .
[101, 9805, 3540, 11514, 2050, 3079, 11282, 2243, 1005, 1055, 2077, 4855, 1996, 4677, 2000, 3647, 4576, 1999, 2687, 2005, 1002, 1016, 1012, 1019, 4551, 1012, 102, 9805, 3540, 11514, 2050, 4149, 11282, 2243, 1005, 1055, 1999, 2786, 2005, 1002, 6353, 2509, 2454, 1998, 2853, 2009, 2000, 3647, 4576, 2005, 1002, 1015, 1012, 1022, 4551, 1999, 2687, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['[CLS]', 'yu', '##ca', '##ip', '##a', 'owned', 'dominic', '##k', "'", 's', 'before', 'selling', 'the', 'chain', 'to', 'safe', '##way', 'in', '1998', 'for', '$', '2', '.', '5', 'billion', '.', '[SEP]', 'yu', '##ca', '##ip', '##a', 'bought', 'dominic', '##k', "'", 's', 'in', '1995', 'for', '$', '69', '##3', 'million', 'and', 'sold', 'it', 'to', 'safe', '##way', 'for', '$', '1', '.', '8', 'billion', 'in', '1998', '.', '[

**Exercise: **

Take element 15 of the training set and tokenize the two sentences separately and as a pair. What’s the difference between the two results?


In [26]:
print(raw_train_dataset["sentence1"][15])
print(raw_train_dataset["sentence2"][15])
input1 = tokenizer(raw_train_dataset["sentence1"][15])
print(input1)
input2 = tokenizer(raw_train_dataset["sentence2"][15])
print(input2)


Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .
Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .
{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [101, 3026, 3580, 2343, 4388, 24049, 1010, 3839, 2132, 1997, 1996, 9722, 1998, 4132, 9340, 12439, 2964, 3131, 1010, 2097, 2599, 1996, 2047, 9178, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [27]:
input = tokenizer(raw_train_dataset["sentence1"][15], raw_train_dataset["sentence2"][15] )
print(input)

{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102, 3026, 3580, 2343, 4388, 24049, 1010, 3839, 2132, 1997, 1996, 9722, 1998, 4132, 9340, 12439, 2964, 3131, 1010, 2097, 2599, 1996, 2047, 9178, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


## Preproccessing all the datasets:

We can tokenize the three subdatasets at once. To do this, we can define the following function "tokenize". This function could be extended to include other preprocessing tasks (removing special words, punctuation, etc). In this tutorial, we are not applying any of these additional preprocessing techniques.

We are also not going to apply padding (that is, adding a special token "pad" for all sequences have the same lenght) because padding all the samples to the maximum length is not efficient.

We will only apply padding in the sequences of a batch, as we only need to pad to the maximum length in that batch, and not the maximum length in the entire dataset. This strategy can save a lot of time and processing power, because the inputs have very variable lengths"

In [28]:
def tokenize(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

The function Dataset.map() allow us to apply the previous function to the three datasets at once.

In [29]:
tokenized_datasets = raw_datasets.map(tokenize, batched=True) # Operate on batches by setting batched=True. The default batch size is 1000
tokenized_datasets

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

We can see that the tokenized dataset keep the previous features (idx, sentence1, sentence2, and label) and also include the new features: idx, input_ids, token_type_ids, and attention_mask.


## Dynamic padding

To apply padding on all sequences that belong to the same batch, we need to know dynamic padding. 


To understand better this, we will create a small batch consisting of eight pair of sentences from the training dataset (we ignore the columns idx, sentence1, and sentence2, because they are not neccesary):


In [30]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}


Let'us to know their lenghts. We can see that lengths range from 32 to 67!!!.

In [31]:
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

**Dynamic padding** will set 67 as the maximum length for this batch. Then, it will only padd the shorter sequences in this batch to the maximum lenght of 67.

Without dynamic padding, all of the samples would have to be padded to the maximum length in the whole dataset, or the maximum length the model can accept. This could be very inneficient!!!


Let's to show how the library transformer allows us to implement Dynamic padding. 
First, we will put together the samples inside a batch. To do this, we can use the following class, DataCollatorWithPadding. We need to save the samples into this collator:

In [32]:
from transformers import DataCollatorWithPadding
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf") # tf
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #pytchorh


batch = data_collator(samples)
#we can see that the batch contains 8 samples and each input sequence has lenght of 67
{k: v.shape for k, v in batch.items()}


{'attention_mask': torch.Size([8, 67]),
 'input_ids': torch.Size([8, 67]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 67])}

## Fine-tuning a model with the Trainer API

Finally!!!, we are going to train our model. To do this, 

Before training our model, we must first define the hyperparameteres used for training and evaluation. 
The TrainingARguments contains these hyperparameteres. No worry!!!, you only need to provide the directoyr where you wanto to save your trained model and their chekpoints (weights). 

We can use the default values for the rest of hyperparameteres (they work pretty well). By the default, we load the arguments from the model test_trainer (https://huggingface.co/xysmalobia/test-trainer), which is a fine-tuned version of bert-base-uncased on the glue dataset. 

In [33]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")
# training_args

Now, we can define our model by using the AutoModelForSequenceClassification class, with two labels:

In [34]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

This warning is telling us that BERT was not  pretrained on classifying pairs of sentences, so the head of the pretrained model has been discarded and a new head suitable for sequence classification has been added instead.

The warnings indicate that some weights were not used (the ones corresponding to the dropped pretraining head) and that some others were randomly initialized (the ones for the new head). 


For this reason, we need to train our own model!!!. Let's go for it!!!


Now we must pass to the Trainer class the following objects: 
- the model, 
- the training_args, 
- the training and validation datasets, 
- data_collator, 
-  tokenizer



In [35]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator, # this line could be skipped because the default data_collector is  DataCollatorWithPadding
    tokenizer=tokenizer,
)


Finally, we can fine-tuning our model:

In [ ]:
trainer.train()


The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1377, training_loss=0.3384241063956688, metrics={'train_runtime': 193.4078, 'train_samples_per_second': 56.895, 'train_steps_per_second': 7.12, 'total_flos': 406183858377360.0, 'train_loss': 0.3384241063956688, 'epoch': 3.0})

The fine-tuning should tak a couple of minutes on a GPU. 
It will only report the training loss every 500 steps, but won't provide any metric that tells us if the model performs well or badly. 



The function Trainer.predict allows us to obtain some predictions from our model. In the following example, we will use the validation data to obtain these predictions. 


In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 408
  Batch size = 8


(408, 2) (408,)


The predict produces a tuple with three fields:
- predictions, produced by the model
- label_ids, which are the gold labels in the validation dataset
- and metrics: the loss function and some temporal metrics (such as how long the model took to predict).

You can see that, predictions provide a probability for each label (predictions is a two-dimensional array with shape 408 x 2 where 408 is the number of intances in the validation dataset). 

To compare the predictions generated by the model with the truth labels in the validation dataset, we need the index with the maximum value on the second axis:



In [ ]:
print(predictions.predictions)


[[-3.6970873   3.5894885 ]
 [ 2.7080908  -2.3771203 ]
 [ 2.189035   -1.8982339 ]
 [-2.74807     2.755175  ]
 [ 2.6027122  -2.543721  ]
 [-3.3365154   3.14389   ]
 [-3.5206838   3.4144459 ]
 [-3.6055214   3.4082823 ]
 [-3.3860018   3.2965882 ]
 [-3.5538824   3.6247203 ]
 [-3.6465755   3.5940151 ]
 [ 2.6831114  -2.4305298 ]
 [ 2.0713327  -1.7795335 ]
 [-2.6889377   2.3802109 ]
 [-3.5524204   3.6184273 ]
 [ 2.4724598  -2.402282  ]
 [-3.4803095   3.563389  ]
 [ 1.7599279  -1.5352368 ]
 [-3.3992178   3.53962   ]
 [ 2.1357286  -1.8835583 ]
 [ 2.6657312  -2.3209956 ]
 [-3.0832655   2.9108207 ]
 [ 1.0651772  -1.0016227 ]
 [-3.1028814   2.8889449 ]
 [-3.4513612   3.4343638 ]
 [-0.06768513 -0.03193824]
 [ 2.4552057  -2.0237153 ]
 [-3.6421363   3.648123  ]
 [-3.5624802   3.3933978 ]
 [-3.580394    3.4993348 ]
 [ 2.587248   -2.3531427 ]
 [-3.5656226   3.5780735 ]
 [-2.942653    2.9107556 ]
 [-3.425567    3.4505794 ]
 [-3.426937    3.4735188 ]
 [-3.3687882   3.4402182 ]
 [ 0.83562124 -0.9692984 ]
 

In [ ]:
import numpy as np
preds = np.argmax(predictions.predictions, axis=-1)
print(preds)
# print(predictions.label_ids)



[1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1 0 0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 0
 0 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1
 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 0 1 1 0
 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0
 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1
 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 0
 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0
 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0
 1]


In [ ]:
# predictions.label_ids
predictions.metrics

{'test_loss': 0.7848561406135559,
 'test_runtime': 1.8546,
 'test_samples_per_second': 219.99,
 'test_steps_per_second': 27.499}

## Loading the metrics for a given dataset (task)

So we should extend the set of metrics to obtain a more informative evaluation of our model. To do this, we have to define a compute_metrics function and pass it to the Trainer. 

Fortunately, the dataset library provides datasets, but also provides the metrics for each dataset. 

In [ ]:
from datasets import load_metric
metric = load_metric("glue", "mrpc")


Now, we can evaluate the model using these metrics. We can see that our model has an accuracy of 85.78% on the validation set and an F1 score of 89.97 (these results may change The exact  as the random initialization of the model head migh

In [ ]:
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8431372549019608, 'f1': 0.8926174496644296}

Let'us a function, compute_metrics, that wrappes all the previous steps. This funcion gets the output of the trainer.predict for a given dataset (for example, the validation dataset). This functions returns an object predictions. We will only use its predictions (each prediction is converted to the index with maximum probability en each pair) and its truth labels.
The function loads the metrics for the dataset used, MRPC. Finally, this metric can computed on the preditions and the truth labels (references). 


In [ ]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

This function, compute-metrics, should be passed to the Trainer object as parameter:

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": 

In [ ]:
trainer.train()


The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ign

TrainOutput(global_step=1377, training_loss=0.3474766932801224, metrics={'train_runtime': 209.2281, 'train_samples_per_second': 52.593, 'train_steps_per_second': 6.581, 'total_flos': 406183858377360.0, 'train_loss': 0.3474766932801224, 'epoch': 3.0})

## Evaluation

We will also provide the results on the test dataset, which has not been used to train the model. 



In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.metrics)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1725
  Batch size = 8


{'test_loss': 0.7723301649093628, 'test_accuracy': 0.823768115942029, 'test_f1': 0.8723761544920234, 'test_runtime': 9.0605, 'test_samples_per_second': 190.388, 'test_steps_per_second': 23.84}


Therefore, we can say that our fine-tuned model for the task of paraphrase identification achieves an accuracy of 82.3% and an F1 of 87.2% on the test dataset de MRPC.

## Lab case:

Fine-tune a model on the GLUE SST-2 dataset, using the data processing you did in section 2.
